In [172]:
import os
import re
import pickle
import nltk
import pandas as pd
import math
import numpy as np
from nltk.stem import SnowballStemmer
from bs4 import BeautifulSoup
directory = 'Documents/'
stopwords = open('Urdu stopwords.txt',encoding='utf-8').read().split("\n")
import warnings
warnings.filterwarnings("ignore")

# Assignment 2

In [156]:
with open('DOCID.pkl', 'rb') as fp:
    DOCID = pickle.load(fp)

In [157]:
with open('TOKENS.pkl', 'rb') as fp:
    TOKENS = pickle.load(fp)

In [158]:
with open('INVERTED_INDEX.pkl', 'rb') as fp:
    INVERTED_INDEX = pickle.load(fp)

In [159]:
with open('INVERTED_INDEX.pkl', 'rb') as fp:
    INVERTED_INDEX_ = pickle.load(fp)

In [160]:
with open('DOCID_.pkl', 'rb') as fp:
    DOCID_ = pickle.load(fp)

In [161]:
def IDF(INVERTED_INDEX,N):
    
    '''This function assigns IDF value of every term'''
    
    for ter in INVERTED_INDEX.keys():
        INVERTED_INDEX[ter].append(1 + math.log10(N//len(INVERTED_INDEX[ter][0])))
    

In [162]:
def IDF_(INVERTED_INDEX,N):
    
    '''This function assigns IDF value of every term'''
    
    for ter in INVERTED_INDEX.keys():
        INVERTED_INDEX[ter].append(math.log10(1 + ((N-len(INVERTED_INDEX[ter][0]))//len(INVERTED_INDEX[ter][0]))))

In [163]:
IDF_(INVERTED_INDEX_,len(DOCID))

In [164]:
IDF(INVERTED_INDEX,len(DOCID))

### Processing Queries 

In [165]:
queries = {1:'مجرم کو پھانسی دے دی گئی'
      ,2:'احتجاجی  دھرنے, جلوس اور ریلیاں'
      ,3:'انسداد تمباکو نوشی کے اقدامات اور آگاہی'
      ,4:'وکلاء کی ہڑتال اور ریلیاں'
      ,5:'خفیہ ایجنسیاں اور جاسوس'
      ,6:'احتساب عدالت کے مقدمات'
      ,7:'بچوں کے خلاف تشدد اور اغوا'
      ,8:'عورتوں پر ظلم و تشدد , قتل اور اغوا'
      ,9:'انسانوں کی اسمگلنگ اور انسداد انسانی سمگلنگ کے اقدامات'
      ,10:'مزدور طبقے کی خوشحالی کے اقدامات'
     }

In [166]:
def Query_processing_with_tfidf_bm25(queries,k1,b):
    
    Score_tfidf_all = {}
    Score_bm25_all = {}
    Score_DS_all = {}
    
    s = 0
    for k in DOCID_.keys():
        s += DOCID_[k][1]

    avgdl = s//len(DOCID_)
    
    Lamda = 5771/(5711 + avgdl)
    OneminusLamda = avgdl/(5711 + avgdl)

    for index_,q in enumerate(queries.keys()):
        Score_tfidf = {}
        Score_bm25 = {}
        Score_DS = {}
        
        
        
        IDS = []
        tokensQ = []
        text = re.sub(re.compile(r'[^\w\s]+'),'',queries[q])
        tokensQ = nltk.word_tokenize(text)
        tokensQ = [i for i in tokensQ if i not in stopwords]
        
        for i in tokensQ:
             IDS.append(TOKENS[i])
                
        for index,j in enumerate(IDS):
            for i in INVERTED_INDEX[j][0].keys():
                if(index == 0):    
                    Score_tfidf[i] = INVERTED_INDEX[j][1] * INVERTED_INDEX[j][0][i][1]
                else:
                    if i in Score_tfidf.keys():
                        Score_tfidf[i] += INVERTED_INDEX[j][1] * INVERTED_INDEX[j][0][i][1]

          
        for index,j in enumerate(IDS):
            for i in INVERTED_INDEX[j][0].keys():  
                K = (k1)*((1-b)+(b*(DOCID_[i][1]/avgdl)))
                if(index == 0):    
                    Score_bm25[i] = (((INVERTED_INDEX_[j][0][i][1])*(k1 + 1))/((INVERTED_INDEX_[j][0][i][1]) + K))*(INVERTED_INDEX_[j][1])                   
                else:
                    if i in Score_bm25.keys():
                        Score_bm25[i] += (((INVERTED_INDEX_[j][0][i][1])*(k1 + 1))/((INVERTED_INDEX_[j][0][i][1]) + K))*(INVERTED_INDEX_[j][1])         

        
        for index,j in enumerate(IDS):  
            back = sum([len(INVERTED_INDEX[j][0][doc_id][0]) for doc_id in INVERTED_INDEX[j][0]]) / s
            for i in INVERTED_INDEX[j][0].keys():  
                prob = INVERTED_INDEX[j][0][i][1]/len(INVERTED_INDEX[j][0][i][0])
                if(index == 0):    
                    Score_DS[i] = ((Lamda)*(prob)) + ((OneminusLamda)*(back))  
                
                else:
                    if i in Score_DS.keys():
                        Score_DS[i] += (Lamda*(prob)) + ((OneminusLamda)*(back))  
        
        
        
        
        Score_tfidf = dict(sorted(Score_tfidf.items(), key=lambda item: item[1], reverse=True))
        Score_bm25 = dict(sorted(Score_bm25.items(), key=lambda item: item[1], reverse=True))
        Score_DS = dict(sorted(Score_DS.items(), key=lambda item: item[1], reverse=True))
        
    
        Score_tfidf_all[q] = Score_tfidf
        Score_bm25_all[q] = Score_bm25
        Score_DS_all[q] = Score_DS
        
         
        rank_DS = 1
        with open('Results_DS.txt', 'a',encoding='utf-8') as file:
            for key,value in Score_DS.items():
                file.write(str(index_)+ " " + str(key) + ' ' + str(rank_DS) + ' ' + str(round(value,ndigits=3)) + ' ' + 'run 1' + '\n')
                rank_DS += 1   
        
        
        rank_tfidf = 1
        with open('Results_TFIDF.txt', 'a',encoding='utf-8') as file:
            for key,value in Score_tfidf.items():
                file.write(str(index_)+ " " + str(key) + ' ' + str(rank_tfidf) + ' ' + str(round(value,ndigits=3)) + ' ' + 'run 1' + '\n')
                rank_tfidf += 1

        
        rank_bm25 = 1
        with open('Results_bm25.txt', 'a',encoding='utf-8') as file:
            for key,value in Score_bm25.items():
                file.write(str(index_)+ " " + str(key) + ' ' + str(rank_bm25) + ' ' + str(round(value,ndigits=3)) + ' ' + 'run 1' + '\n')
                rank_bm25 += 1
                
                         
        

In [167]:
Query_processing_with_tfidf_bm25(queries,1.2,0.75)